In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/Users/fdangelo/PycharmProjects/myRBM")
import numpy as np 
import scipy.constants as cs
import deepdish as dd
from sklearn.preprocessing import Binarizer
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import tensorflow as tf
output_notebook()

Loading BokehJS ...

In [4]:
datah5 = dd.io.load('/Users/fdangelo/PycharmProjects/myRBM/data/ising/ising_data_complete.hdf5')

In [5]:
#Take spin up as standard configuration
keys = list(datah5.keys())
datah5_norm = {}
data_bin = {}
binarizer = Binarizer(threshold=0)
for key in keys:
    datah5_norm[key] = np.array([np.where(np.sum(slice)<0,-slice,slice) for slice in datah5[key]])
    data_bin[key] = np.array([binarizer.fit_transform(slice) for slice in datah5_norm[key]]).reshape(datah5_norm[key].shape[0],-1).astype(np.float32)

In [6]:
magn=[]
std = []
for key in keys:
    magn.append(np.array([np.average(slice) for slice in datah5_norm[key]]))
    std.append(np.array([np.std(slice) for slice in datah5_norm[key]]))

In [8]:
colors = [ 'yellow','green', 'orange','navy','grey','red']
output_notebook()
p = figure(plot_width=1000, plot_height=400,y_range=(0, 1.01))
colours = []
#for idx,i in enumerate(magn): 
 #   p.circle(np.arange(magn[0].shape[0]),i, size=7, line_color="navy", fill_color=colors[idx], fill_alpha=0.7, legend = keys[idx])

for en,pt in enumerate(keys):
    p.circle(np.arange(magn[en].shape[0])[:1000],magn[en][:1000], size=7, line_color="navy", fill_color=colors[en], fill_alpha=0.7, legend = keys[en])

p.yaxis.axis_label = "Magnetization"
p.xaxis.axis_label = "Samples"
p.legend.location = "bottom_right"
p.legend.click_policy="hide"
show(p) # show the results

Loading BokehJS ...

In [20]:
temp = [1.000000/2.269184, 2.186995/2.269184, 2.261435/2.269184,2.268900/2.269184,2.269184/2.269184, 3.000000/2.269184]
T = np.linspace(0.1,3,100)
teor_magn = np.nan_to_num(np.power((1-np.power(np.sinh(2/(T)),-4)),0.125))

/Users/fdangelo/anaconda3/envs/RBM/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# create a new plot with default tools, using figure
from bokeh.models import ColumnDataSource, Whisker
p = figure(plot_width=600, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(temp,mean_magn_temp, size=7, line_color="navy", fill_color="green", fill_alpha=0.5, legend = 'Simulated conf')
p.line(T/2.269, teor_magn,line_color="red", line_width=3, line_alpha=0.6, legend='Theoretical curve' )
source_error = ColumnDataSource(data=dict(base=temp, lower=np.asarray(mean_magn_temp)-np.asarray(std_magn_temp), upper=np.asarray(mean_magn_temp)+np.asarray(std_magn_temp)))
p.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)

p.yaxis.axis_label = "Magnetization"
p.xaxis.axis_label = "T/Tcritic"
p.legend.location = "bottom_left"
show(p) # show the results

In [60]:
#Principal component on ising dataset: 
from sklearn.decomposition import PCA
import itertools
import pandas as pd 
import matplotlib.pyplot as plt 

pca = PCA(n_components=2, svd_solver='full')
class_labels = np.asarray(list(itertools.chain.from_iterable(itertools.repeat(x, 5000) for x in range(0,len(keys)))))
data = data_bin[keys[0]]
for temperature in keys[1:]:
    data = np.concatenate([data,data_bin[temperature]])

In [61]:
principalComponents = pca.fit_transform(data)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])


In [62]:
principalDf['target'] = class_labels

In [63]:
print(pca.explained_variance_ratio_)

[0.15537995 0.02265481]


In [80]:
colors = [ 'yellow','green', 'orange','navy','grey','red']
output_notebook()
p2 = figure(plot_width=800, plot_height=800)

targets = [0,1,2,3,4,5]
for target, color in zip(targets,colors): 
    indicesToKeep = principalDf['target'] == target
    p2.circle(principalDf.loc[indicesToKeep, 'principal component 1']
               , principalDf.loc[indicesToKeep, 'principal component 2'], size=7, line_color="navy", fill_color=color, fill_alpha=0.7, legend = keys[target])
    

p2.yaxis.axis_label = "Principal component 2"
p2.xaxis.axis_label = "Principal component 1"
p2.legend.location = "bottom_right"
p2.legend.click_policy="hide"
show(p2) # show the results

Loading BokehJS ...

In [79]:
PCA_mnist = pca.fit_transform(x_train_binary)
PCADf_mnist = pd.DataFrame(data = PCA_mnist
             , columns = ['principal component 1', 'principal component 2'])
PCADf_mnist['target'] = y_train

In [78]:
colors = [ 'yellow','green', 'orange','navy','grey','red','black','violet', 'white', 'brown']
output_notebook()
p3 = figure(plot_width=800, plot_height=800)

targets = [0,1,2,3,4,5,6,7,8,9]
for target, color in zip(targets,colors): 
    indicesToKeep = PCADf_mnist['target'] == target
    p3.circle(PCADf_mnist.loc[indicesToKeep, 'principal component 1']
               , PCADf_mnist.loc[indicesToKeep, 'principal component 2'], size=7, line_color='navy', fill_color=color, fill_alpha=0.7, legend = str(target))
    
p3.yaxis.axis_label = "Principal component 2"
p3.xaxis.axis_label = "Principal component 1"
p3.legend.location = "bottom_right"
p3.legend.click_policy="hide"
show(p3) # show the results

Loading BokehJS ...

In [81]:
x = 5 
print(type(x))

<class 'int'>


In [82]:
x = 5.0
print(type(x))

<class 'float'>
